In [879]:
import pandas as pd
import numpy as np

In [880]:
train = pd.read_csv('dataset/train_u6lujuX_CVtuZ9i.csv')
train.Loan_Status = train.Loan_Status.map({'Y':1,'N':0})
y_train = train.pop('Loan_Status').values

train['source']='train'
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,source
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,train
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,train
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,train
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,train
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,train


In [882]:
test = pd.read_csv('dataset/test_Y3wMUE5_7gLdaTN.csv')
test['source']='test'
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,source
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban,test
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban,test
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban,test
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban,test
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban,test


In [883]:
df = pd.concat([train, test],ignore_index=True)
df = df.drop(['Loan_ID'],axis=1)
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,source
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,train
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,train
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,train
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,train
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,train


In [884]:
df.isnull().sum()

Gender               24
Married               3
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
source                0
dtype: int64

In [885]:
from scipy.stats import mode
df.Credit_History = df.Credit_History.fillna(mode(df.Credit_History).mode[0])
df.Dependents = df.Dependents.fillna(mode(df.Dependents).mode[0])
df.Gender = df.Gender.fillna(mode(df.Gender).mode[0])
df.LoanAmount = df.LoanAmount.fillna(df.LoanAmount.mean())
df.Loan_Amount_Term = df.Loan_Amount_Term.fillna(df.Loan_Amount_Term.mean())
df.Married = df.Married.fillna(mode(df.Married).mode[0])
df.Self_Employed = df.Self_Employed.fillna(mode(df.Self_Employed).mode[0])
df.isnull().sum()

/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
source               0
dtype: int64

In [886]:
df.Credit_History = df.Credit_History.astype(int)
df.Dependents = df.Dependents.replace('3+','3')

In [887]:
def LAClass(x):
    if x < 101.0:
        return 1
    elif x >= 101.0 and x < 128.0:
        return 2
    elif x >= 128.0 and x < 160.0:
        return 3
    else:
        return 4

df['Loan_Category'] = df['LoanAmount'].apply(lambda x: LAClass(x)) 
df = df.drop('LoanAmount', axis=1)

In [888]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Education','Gender','Married','Property_Area','Self_Employed','Dependents']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History,Property_Area,source,Loan_Category
0,1,0,0,0,0,5849,0.0,360.000000,1,2,train,3
1,1,1,1,0,0,4583,1508.0,360.000000,1,0,train,3
2,1,1,0,0,1,3000,0.0,360.000000,1,2,train,1
3,1,1,0,1,0,2583,2358.0,360.000000,1,2,train,2
4,1,0,0,0,0,6000,0.0,360.000000,1,2,train,3
5,1,1,2,0,1,5417,4196.0,360.000000,1,2,train,4
6,1,1,0,1,0,2333,1516.0,360.000000,1,2,train,1
7,1,1,3,0,0,3036,2504.0,360.000000,0,1,train,3
8,1,1,2,0,0,4006,1526.0,360.000000,1,2,train,4
9,1,1,1,0,0,12841,10968.0,360.000000,1,1,train,4


In [889]:
df.dtypes

Gender                 int64
Married                int64
Dependents             int64
Education              int64
Self_Employed          int64
ApplicantIncome        int64
CoapplicantIncome    float64
Loan_Amount_Term     float64
Credit_History         int64
Property_Area          int64
source                object
Loan_Category          int64
dtype: object

In [890]:
train = df.loc[df['source']=="train"]
test = df.loc[df['source']=="test"]

train = train.drop('source',axis=1)
test = test.drop('source',axis=1)

X_train = train.loc[:,:]
X_test = test.loc[:,:]

print(train.head())
print(test.head())

   Gender  Married  Dependents  Education  Self_Employed  ApplicantIncome  \
0       1        0           0          0              0             5849   
1       1        1           1          0              0             4583   
2       1        1           0          0              1             3000   
3       1        1           0          1              0             2583   
4       1        0           0          0              0             6000   

   CoapplicantIncome  Loan_Amount_Term  Credit_History  Property_Area  \
0                0.0             360.0               1              2   
1             1508.0             360.0               1              0   
2                0.0             360.0               1              2   
3             2358.0             360.0               1              2   
4                0.0             360.0               1              2   

   Loan_Category  
0              3  
1              3  
2              1  
3              2  
4  

In [891]:
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
import xgboost as xgb

clf1 = RandomForestClassifier(n_estimators = 106, random_state=1)
clf2 = xgb.XGBClassifier(max_depth=50, min_child_weight=1,  n_estimators=200, n_jobs=-1 , verbose=1,learning_rate=0.16)
clf3 = DecisionTreeClassifier(max_depth = 10)
lr = LogisticRegression()


sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], meta_classifier=lr)

sclf.fit(X_train,y_train)


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


StackingClassifier(average_probas=False,
          classifiers=[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_f...       min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')],
          meta_classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0)

In [892]:
y_pred = sclf.predict(X_test)
y_pred = pd.Series(y_pred.astype(int))
print(y_pred)
y_pred = y_pred.map({1.:'Y',0.:'N'}).values
print(y_pred)
#y_pred = lb.inverse_transform(y_pred)

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      1
10     0
11     1
12     1
13     0
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     0
26     1
27     1
28     1
29     1
      ..
337    1
338    1
339    0
340    1
341    1
342    0
343    1
344    1
345    1
346    0
347    1
348    1
349    1
350    1
351    1
352    1
353    1
354    0
355    1
356    1
357    1
358    1
359    1
360    1
361    1
362    1
363    1
364    1
365    1
366    1
Length: 367, dtype: int64
['Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'N' 'Y' 'N' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y'
 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'N' 'Y'

In [893]:
sample_file = pd.read_csv('dataset/Sample_Submission_ZAuTl8O_FK3zQHh.csv')
sample_file['Loan_Status'] = y_pred
sample_file.to_csv('loan_submission1.csv', index=False)